# Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import config
from joblib import dump, load
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from GMHI import GMHI

# Load data

In [2]:
X = pd.read_csv(os.path.join(config.CLEAN_DIR, "taxonomic_features.csv"), index_col=[0, 1])
y = pd.read_csv(os.path.join(config.CLEAN_DIR, "is_healthy.csv"), index_col=[0, 1])
display(X.head())
display(y.head())

k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                            
GMHI-10  SAMN03283239        0.000000                     0.000000   
         SAMN03283266        0.000000                     0.000000   
         SAMN03283281        0.009764                     0.009764   
         SAMN03283245        0.106067                     0.106067   
         SAMN03283294        0.000000                     0.000000   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID Sample Accession                                                   
GMHI-10  SAMN03283239                                            0.000000   
         SAMN03283266                                            0.000000   
         SAMN03283281                                            0.009764   
         SAMN03283245                                            0.106067   
         SAMN03283294                                            0.000000   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID Sample Accession                                                                         
GMHI-10  SAMN03283239                                               0.000000                      
         SAMN03283266                                               0.000000                      
         SAMN03283281                                               0.009764                      
         SAMN03283245                                               0.106067                      
         SAMN03283294                                               0.000000                      

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID Sample Accession                                                                                                
GMHI-10  SAMN03283239                                               0.000000                                             
         SAMN03283266                                               0.000000                                             
         SAMN03283281                                               0.009764                                             
         SAMN03283245                                               0.106067                                             
         SAMN03283294                                               0.000000                                             

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter  \
Study_ID Sample Accession                                                                                                                      
GMHI-10  SAMN03283239                                               0.000000                                                                   
         SAMN03283266                                               0.000000                                                                   
         SAMN03283281                                               0.009764                                                                   
         SAMN03283245                                               0.072453                                                                   
         SAMN03283294                                               0.000000                                                                   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-10  SAMN03283239                                               0.000000              

PHENOTYPE:Healthy_Nonhealthy
Study_ID Sample Accession                              
GMHI-10  SAMN03283239                              True
         SAMN03283266                              True
         SAMN03283281                              True
         SAMN03283245                              True
         SAMN03283294                              True

# Train Models!

## Train GMHI model on entire dataset

In [3]:
gmhi = GMHI(use_shannon=True, theta_f=1.4, theta_d=0.1)

# use only non-viral species
X_GMHI = X.copy()
X_GMHI = X_GMHI[list(filter(lambda x : "s__" in x and "virus" not in x, X_GMHI.columns))]

gmhi.fit(X_GMHI, y)
GMHI_scores = pd.DataFrame(gmhi.decision_function(X_GMHI), index=y.index, columns=["GMHI"])
display(GMHI_scores)
print("GMHI balanced_accuracy on training set:", balanced_accuracy_score(y, GMHI_scores > 0))
idx = (abs(GMHI_scores) > 0.1).values
print("GMHI balanced_accuracy on training set |GMHI| > 0.1:", balanced_accuracy_score(y[idx], GMHI_scores[idx] > 0))

GMHI
Study_ID Sample Accession          
GMHI-10  SAMN03283239      3.847288
         SAMN03283266      1.769726
         SAMN03283281      3.365736
         SAMN03283245      4.346680
         SAMN03283294      1.407946
...                             ...
P98      SAMEA6104852      1.890616
         SAMEA6104851      0.519115
         SAMEA6104850      1.548641
         SAMEA6104885     -0.087274
         SAMEA6104858      1.562640

[8869 rows x 1 columns]

GMHI balanced_accuracy on training set: 0.7085746568500715
GMHI balanced_accuracy on training set |GMHI| > 0.1: 0.7174770212326024


## Train GMHI2 model on entire dataset

In [4]:
gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=config.REGULARIZATION, class_weight="balanced")
X_GMHI2 = X > config.PRESENCE_CUTOFF
gmhi2.fit(X_GMHI2, y.values.flatten())
GMHI2_scores = pd.DataFrame(gmhi2.decision_function(X_GMHI2), index=y.index, columns=["GMHI2"])
display(GMHI2_scores)
print("GMHI2 balanced_accuracy on training set:", balanced_accuracy_score(y, GMHI2_scores > 0))
idx = (abs(GMHI2_scores) > 0.1).values
print("GMHI2 balanced_accuracy on training set |GMHI2| > 0.1:", balanced_accuracy_score(y[idx], GMHI2_scores[idx] > 0))


GMHI2
Study_ID Sample Accession          
GMHI-10  SAMN03283239      1.911264
         SAMN03283266      0.292261
         SAMN03283281      1.012555
         SAMN03283245      1.193557
         SAMN03283294      2.163656
...                             ...
P98      SAMEA6104852      0.684386
         SAMEA6104851     -0.797367
         SAMEA6104850      1.058772
         SAMEA6104885      0.196350
         SAMEA6104858      1.055208

[8869 rows x 1 columns]

GMHI2 balanced_accuracy on training set: 0.7873102566987858
GMHI2 balanced_accuracy on training set |GMHI2| > 0.1: 0.8009437551850782


# Export stuff

## Export GMHI scores

In [5]:
GMHI_scores.to_csv(os.path.join(config.PREDICTION_DIR, "GMHI_scores.csv"))

## Export GMHI2 scores and serialize model

In [6]:
GMHI2_scores.to_csv(os.path.join(config.PREDICTION_DIR, "GMHI2_scores.csv"))

In [7]:
dump(gmhi2, os.path.join(config.MODEL_DIR, 'gmhi2_model.joblib'))

['../output/models/gmhi2_model.joblib']